In [11]:
from gmip.utils import  analytical_tpr,fn_dong
from ipywidgets import widgets, interactive, HBox, VBox, Layout, FloatText, IntText, Text, Button, Output
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import trapz 
import math

In [17]:
def plot_x(mu=1):
    w = np.linspace(0, 1, 100)
    plt.ylabel("TPR")
    plt.xlabel("FPR")
    plt.xlim(0, 1)
    plt.ylim(0, 1)
  
    plt.grid(color='0.6')
    plt.gcf().set_size_inches(3.0,3.0)
    plt.plot(w,analytical_tpr(w,mu))
    plt.title("Trade-off curve")

def calc_privacy_lvl(C, tau, T, n, N, d, K):
    #print(C, tau, T, n, N, d, K)
    n_eff = n + (n*n*tau*tau)/(C*C)
    #print("neff=", n_eff)
    #mu_step = (d+(2*n_eff-1)*K)/(n_eff*math.sqrt(2*d + 4*n_eff*K))
    mu_step = (d+(2*n-1)*K)/(n_eff*math.sqrt(2*d + 4*((n*n)/n_eff)*K))
    #print("mu_step=", mu_step)
    c = (n*math.sqrt(T))/N
    mu_tot =  c*fn_dong(mu_step)
    return mu_tot

def floater(C):
    try:
        # Try converting C to float. 'inf' will become float('inf'), and numeric strings will become floats.
        C = math.inf if C == 'inf' else float(C)
    except ValueError:
        # If C cannot be converted to float (meaning it's not a numeric string or 'inf'), default to 1.0
        C = 1.0
    return C

In [21]:
def wrapper(C, tau, T, n, N, d, K):
    C=floater(C)
    result = calc_privacy_lvl(C, tau, T, n, N, d, K)
    fpr_values = np.linspace(0, 1, 100)
    tpr_values = [analytical_tpr(fpr, result) for fpr in fpr_values]
    au_roc = trapz(tpr_values, fpr_values)
    tpr_at_001 = tpr_values[np.argmin(np.abs(fpr_values - 0.01))]
    tpr_at_005 = tpr_values[np.argmin(np.abs(fpr_values - 0.05))]
    acc = 0.5*(tpr_values + (1-fpr_values))
    print(f"Privacy Level: {result}")
    print(f"AU-ROC: {au_roc}")
    print(f"TPR@0.01: {tpr_at_001}")
    print(f"TPR@0.05: {tpr_at_005}")
    print('Accuracy:',np.max(acc))
    plot_x(result)


    
    
display(Markdown('<strong>{}</strong><br/>'.format("How private is your model?")))
C_widget = widgets.Text(value='inf', description='Cropping threshold (C):', placeholder='Enter a number or inf')
tau_widget = widgets.FloatText(value=0, description='tau:')#noise added to gradient 
T_widget = widgets.IntText(value=10, description='T:')# Number of SGD steps
n_widget = widgets.IntText(value=10, description='n:') # Size of the dataset HOW MUCH
N_widget = widgets.IntText(value=10, description='N:')# Number of the dataset
d_widget = widgets.IntText(value=2, description='d:')# Number of parameters of the model
K_widget = widgets.FloatText(value=1.0, description='K:')# Data-dependent susceptibility

ui = widgets.interactive(wrapper, C=C_widget, tau=tau_widget, T=T_widget, n=n_widget, N=N_widget, d=d_widget, K=K_widget)
display(ui)
#widgets.interact(plot_x,mu=(0,5,0.1))

#inf string change in func


<strong>How private is your model?</strong><br/>

interactive(children=(Text(value='inf', description='Cropping threshold (C):', placeholder='Enter a number or …